In the following two code cells, we inquire about our current compute environment i.e., runtime. Using Colab Pro (not required), we would like to have a high-end GPU e.g., T4, and/or TPU (tensor processor) e.g., v5e-1, for hardware acceleration, as well as a large amount of RAM (> 20GB) for machine learning processing.

In [ ]:
# Here we check our runtime's GPU and/or TPU type
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Nov 28 16:24:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   34C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Here we check our runtime's RAM amount
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')
# Sample output from this cell when the runtime h/w acceleration is selected v5e-1
#Your runtime has 54.8 gigabytes of available RAM
#
#You are using a high-RAM runtime!

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In the following code cell, we read and load the preprocessed MS Excel file prepared and originally used by Tarik Oguz. This is temporary and for confirmation purposes only, until we describe and implement the precise steps to reproduce this dataset i.e., the MS Excel file contents, starting with the original series data obtained from https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE137140.

In [ ]:
import pandas as pd
from scipy import stats

# The DataFrame is named 'df'
# The column containing patient classification information
#    e.g., "disease state: Non-cancer control", "disease state: Lung cancer, pre-operation",
#    and "disease state: Lung cancer, post-operation", is named '!Sample_characteristics_ch1'

# Install openpyxl if not already installed
# This seems to be needed for at least a TPU accelerated runtime environment
!pip install openpyxl

# Read the MS Excel file into a Pandas DataFrame
df = pd.read_excel('/content/COLAB.xlsx')

# This cell took ~1m to run including openpyxl installation using a TPU and high RAM
# This cell took ~2m to run not including openpyxl installation using a Tesla T4 GPU and high RAM
#
# When run for the first time, it generated the following output:
#Collecting openpyxl
#  Downloading openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
#Collecting et-xmlfile (from openpyxl)
#  Downloading et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
#Downloading openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
#   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 8.5 MB/s eta 0:00:00
#Downloading et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
#Installing collected packages: et-xmlfile, openpyxl
#Successfully installed et-xmlfile-2.0.0 openpyxl-3.1.5
#
# In consequent runs, the following output is generated:
#Requirement already satisfied: openpyxl in /usr/local/lib/python3.12/dist-packages (3.1.5)
#Requirement already satisfied: et-xmlfile in /usr/local/lib/python3.12/dist-packages (from openpyxl) (2.0.0)

Here, we describe, step-by-step, the process needed to reproduce the dataset prepared and originally used by Tarik Oguz.


1.   At https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE137140, download ('Download family' section at the bottom of the web page), the archive file option 'Series Matrix File(s)' (with format TXT). This will lead to the download of a file with name GSE137140_series_matrix.txt.gz.
2.   Extract the contents of the downloaded archive file GSE137140_series_matrix.txt.gz. This will lead to the generation of a text file with name GSE137140_series_matrix.txt. (Making this file read-only is a good idea so as to keep the original copy intact.)
3.   Open the generated file GSE137140_series_matrix.txt in a text editor and save it with the new name GSE137140_series_matrix_work.txt. This will be our working copy.
4.   Delete the following 36 preamble lines from the file GSE137140_series_matrix_work.txt and save it.

!Series_title	"Blood test using serum microRNAs can discriminate lung cancer from non-cancer"

!Series_geo_accession	"GSE137140"

!Series_status	"Public on Mar 30 2020"

!Series_submission_date	"Sep 09 2019"

!Series_last_update_date	"Mar 30 2020"

!Series_pubmed_id	"32193503"

!Series_summary	"A serum miRNA combination could be a powerful classifier for the detection of patients with lung cancers."

!Series_overall_design	"Circulting microRNA profiles of 3924 samples consisted of 1566 preoperative lung cancer, 180 postoperative lung cancer and 1774 non-cancer controls."

!Series_type	"Expression profiling by array"

!Series_contributor	"Keisuke,,Asakura"

!Series_contributor	"Tsukasa,,Kadota"

!Series_contributor	"Juntaro,,Matsuzaki"

!Series_contributor	"Yukihiro,,Yoshida"

!Series_contributor	"Yusuke,,Yamamoto"

!Series_contributor	"Kazuo,,Nakagawa"

!Series_contributor	"Makiko,,Ichikawa"

!Series_contributor	"Satoko,,Takizawa"

!Series_contributor	"Yoshiaki,,Aoki"

!Series_contributor	"Hiromi,,Sakamoto"

!Series_contributor	"Ken,,Kato"

!Series_contributor	"Shun-ichi,,Watanabe"

!Series_contributor	"Takahiro,,Ochiya"

!Series_sample_id	"GSM4067570 GSM4067571 GSM4067572 ... (This line is a particularly long line which will look like multiple lines due to wrap-arounds in your editor.)

!Series_contact_name	"Satoko,,Takizawa"

!Series_contact_email	"satoko.takizawa.f3@mail.toray"

!Series_contact_institute	"Toray industries inc"

!Series_contact_address	"kamakurashi"

!Series_contact_city	"Kanagawa"

!Series_contact_zip/postal_code	"248-8555"

!Series_contact_country	"Japan"

!Series_supplementary_file	"ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE137nnn/GSE137140/suppl/GSE137140_RAW.tar"

!Series_platform_id	"GPL21263"

!Series_platform_taxid	"9606"

!Series_sample_taxid	"9606"

!Series_relation	"BioProject: https://www.ncbi.nlm.nih.gov/bioproject/PRJNA564633"

(This last line is an empty line.)

5.   In the saved file GSE137140_series_matrix_work.txt, move to line # 34, listed below, delete it, and save the file again.

!series_matrix_table_begin

6.   In the saved file GSE137140_series_matrix_work.txt, move to the very end of the file, delete the last line, listed below, and save the file again.

!series_matrix_table_end

7.   In the saved file GSE137140_series_matrix_work.txt, move to the very beginning of the six lines identified below (with their line numbers), and perform the changes listed below to the first string on each line, and save the file again. This is crucial to avoid data loss, as duplicate column names lead to data loss while data is being read from a text formatted table and converted into a Pandas dataframe.

Line 10, beginning:
!Sample_characteristics_ch1 -> revise as -> !Sample_characteristics_1_ch1

Line 11, beginning:
!Sample_characteristics_ch1 -> revise as -> !Sample_characteristics_2_ch1

Line 12, beginning:
!Sample_characteristics_ch1 -> revise as -> !Sample_characteristics_3_ch1

Line 13, beginning:
!Sample_characteristics_ch1 -> revise as -> !Sample_characteristics_4_ch1

Line 21, beginning:
!Sample_description -> revise as -> !Sample_description_1

Line 22, beginning:
!Sample_description -> revise as -> !Sample_description_2

8.   Our raw data file is in a complete, rectangular format right now and ready to be read by the following code cell. Each row is either for an attribute (feature) specified for each sample i.e., patient, or for a specific miRNA expression measured and reported for each sample. Each column corresponds to a unique sample i.e., patient, and contains the attributes (features) and measured miRNA expressions specific to that sample.

In [ ]:
import csv

data = {}
with open("/content/GSE137140_series_matrix_work.txt", "r", newline="") as f:
    reader = csv.reader(f, delimiter="\t")  # or ',' depending on your file
    for row in reader:
        if not row:
            continue  # skip empty lines
        key = row[0]  # first element = descriptor
        values = []
        for val in row[1:]:
            # Try to convert to float if numeric
            try:
                values.append(float(val))
            except ValueError:
                # Keep as string (quotes already handled by csv)
                values.append(val)
        data[key] = values

#print(data)
# In a TPU accelerated high RAM environment, this cell took ~1s to execute.
# In a GPU accelerated high RAM environment, this cell took ~3s-4s to execute.

In [ ]:
# Some exploratory data analysis on data
# first convert data into a pandas dataframe
df1 = pd.DataFrame(data)

In [ ]:
df1.head()
# One thing to note is the following. The conversion from the text formatted,
#    preprocessed, rectangular series data through a csv read into a pandas
#    dataframe, resulted in the transposition of the data set. Now, the rows
#    correspond to unique samples i.e., patients, and the columns correspond
#    to individual attributes (features) or specific miRNAs with measured
#    expressions with a descriptive column name (title, label) .

,!Sample_title,!Sample_geo_accession,!Sample_status,!Sample_submission_date,!Sample_last_update_date,!Sample_type,!Sample_channel_count,!Sample_source_name_ch1,!Sample_organism_ch1,!Sample_characteristics_1_ch1,...,MIMAT0031893,MIMAT0032026,MIMAT0032029,MIMAT0032110,"MIMAT0032114, MIMAT0032115",MIMAT0032116,MIMAT0033692,MIMAT0035542,MIMAT0035703,MIMAT0035704
0,BC0921,GSM4067570,Public on Mar 30 2020,Sep 09 2019,Mar 30 2020,RNA,1.0,Serum,Homo sapiens,disease state: Non-cancer control,...,-0.152745,-0.152745,5.312396,-0.152745,-0.152745,7.239878,3.096798,-0.152745,-0.152745,0.252979
1,BC0962,GSM4067571,Public on Mar 30 2020,Sep 09 2019,Mar 30 2020,RNA,1.0,Serum,Homo sapiens,disease state: Non-cancer control,...,-1.240936,1.314120,6.919166,-1.240936,-1.240936,6.827426,4.358254,2.766291,-1.240936,-1.240936
2,BC0963,GSM4067572,Public on Mar 30 2020,Sep 09 2019,Mar 30 2020,RNA,1.0,Serum,Homo sapiens,disease state: Non-cancer control,...,1.530600,-1.353647,6.632532,-1.531527,-1.531527,5.173705,4.844177,1.146725,-1.531527,-1.531527
3,BC0973,GSM4067573,Public on Mar 30 2020,Sep 09 2019,Mar 30 2020,RNA,1.0,Serum,Homo sapiens,disease state: Non-cancer control,...,5.404108,0.530399,7.903956,4.832157,0.530399,6.444462,4.290578,0.530399,4.570456,4.113035
4,BC0977,GSM4067574,Public on Mar 30 2020,Sep 09 2019,Mar 30 2020,RNA,1.0,Serum,Homo sapiens,disease state: Non-cancer control,...,0.853141,3.142655,8.361825,0.853141,0.853141,3.301567,0.853141,0.853141,0.853141,0.853141


In [ ]:
df1.tail()

,!Sample_title,!Sample_geo_accession,!Sample_status,!Sample_submission_date,!Sample_last_update_date,!Sample_type,!Sample_channel_count,!Sample_source_name_ch1,!Sample_organism_ch1,!Sample_characteristics_1_ch1,...,MIMAT0031893,MIMAT0032026,MIMAT0032029,MIMAT0032110,"MIMAT0032114, MIMAT0032115",MIMAT0032116,MIMAT0033692,MIMAT0035542,MIMAT0035703,MIMAT0035704
3919,LK2762,GSM4071489,Public on Mar 30 2020,Sep 09 2019,Mar 30 2020,RNA,1.0,Serum,Homo sapiens,"disease state: Lung cancer, post-operation",...,-1.041624,-1.041624,5.765011,-1.041624,-1.041624,5.104661,-1.041624,-1.041624,-1.041624,-1.041624
3920,LK2758,GSM4071490,Public on Mar 30 2020,Sep 09 2019,Mar 30 2020,RNA,1.0,Serum,Homo sapiens,"disease state: Lung cancer, post-operation",...,-1.107796,-1.107796,4.555224,-1.107796,-1.107796,4.820332,5.766157,-1.107796,-1.107796,-1.107796
3921,LK2760,GSM4071491,Public on Mar 30 2020,Sep 09 2019,Mar 30 2020,RNA,1.0,Serum,Homo sapiens,"disease state: Lung cancer, post-operation",...,-1.478147,-1.478147,7.017009,-1.478147,-1.478147,4.676502,5.347159,2.964857,-1.478147,-1.200794
3922,LK2766,GSM4071492,Public on Mar 30 2020,Sep 09 2019,Mar 30 2020,RNA,1.0,Serum,Homo sapiens,"disease state: Lung cancer, post-operation",...,4.422162,-0.660181,6.546635,-0.660181,-0.660181,6.993437,5.845910,5.592984,-0.660181,-0.660181
3923,LK2761,GSM4071493,Public on Mar 30 2020,Sep 09 2019,Mar 30 2020,RNA,1.0,Serum,Homo sapiens,"disease state: Lung cancer, post-operation",...,-1.511059,-1.511059,7.377992,-1.511059,-1.511059,5.236518,-1.511059,-1.511059,-1.511059,-1.511059


In [ ]:
df1.info()
# First output prior to the issue regarding duplicate column names being fixed
#<class 'pandas.core.frame.DataFrame'>
#RangeIndex: 3924 entries, 0 to 3923
#Columns: 2595 entries, !Sample_title to MIMAT0035704
#dtypes: float64(2568), object(27)
#memory usage: 77.7+ MB
#
# Second output consequent to the fix of the issue regarding duplicate column names
# As expected, 4 columns have been added
#<class 'pandas.core.frame.DataFrame'>
#RangeIndex: 3924 entries, 0 to 3923
#Columns: 2599 entries, !Sample_title to MIMAT0035704
#dtypes: float64(2568), object(31)
#memory usage: 77.8+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3924 entries, 0 to 3923
Columns: 2599 entries, !Sample_title to MIMAT0035704
dtypes: float64(2568), object(31)
memory usage: 77.8+ MB


In [ ]:
# The following is much likely to be useful once the dataset is partitioned into negative and positive classes
#df1.describe()

In [ ]:
df1.shape
# before fix: (3924, 2595)
# after fix: (3924, 2599)

(3924, 2599)

In [ ]:
df1.columns

Index(['!Sample_title', '!Sample_geo_accession', '!Sample_status',
       '!Sample_submission_date', '!Sample_last_update_date', '!Sample_type',
       '!Sample_channel_count', '!Sample_source_name_ch1',
       '!Sample_organism_ch1', '!Sample_characteristics_1_ch1',
       ...
       'MIMAT0031893', 'MIMAT0032026', 'MIMAT0032029', 'MIMAT0032110',
       'MIMAT0032114, MIMAT0032115', 'MIMAT0032116', 'MIMAT0033692',
       'MIMAT0035542', 'MIMAT0035703', 'MIMAT0035704'],
      dtype='object', length=2599)

In [ ]:
df1.index

RangeIndex(start=0, stop=3924, step=1)

In [ ]:
# Based on the following query of column names:
# - miRNA identifiers start at column # 34 (0 based counter) and go up to column # 2598
#   hence, there are a total of 2598-34+1=2565 miRNA identifiers (in as many columns)
# - Initially, we would like to construct a dataframe only with columns [9, 34:2598 ]
#   i.e., 2566 columns and all 3924 rows
#   Column # 9 provides the class label for each sample i.e., patient, as one of (non-cancer, cancer_pre-op, cancer_post-op)
#   Remaining columns provide measured expression values of 2565 specific miRNAs for each sample i.e., patient
#   3924 rows correspond to unique samples i.e., patients, each within one of the
#   three categories (non-cancer, cancer_pre-op, cancer_post-op)
# - As discussed later, we will eventually also remove samples of the category cancer_post-op
# - However, before that we would like to review the values in the columns [0:8, 10:33] we are about to delete
df1.iloc[0,0:35]

,0
!Sample_title,BC0921
!Sample_geo_accession,GSM4067570
!Sample_status,Public on Mar 30 2020
!Sample_submission_date,Sep 09 2019
!Sample_last_update_date,Mar 30 2020
!Sample_type,RNA
!Sample_channel_count,1.0
!Sample_source_name_ch1,Serum
!Sample_organism_ch1,Homo sapiens
!Sample_characteristics_1_ch1,disease state: Non-cancer control


In [ ]:
# Review of the data in the columns we are about to delete
for i in range(35):
  print("Column index: ", i, "; Column name: ", df1.columns[i])
  print("Count of unique values in the column: ", df1.iloc[:,i].nunique())
  if df1.iloc[:,i].nunique() < 10:
    print("The unique value(s) in the column: ", df1.iloc[:,i].unique())
  else:
    print("A sample value from the column: ", df1.iloc[0,i])
  print(" ") # space for easy readibility

Column index:  0 ; Column name:  !Sample_title
Count of unique values in the column:  3924
A sample value from the column:  BC0921
 
Column index:  1 ; Column name:  !Sample_geo_accession
Count of unique values in the column:  3924
A sample value from the column:  GSM4067570
 
Column index:  2 ; Column name:  !Sample_status
Count of unique values in the column:  1
The unique value(s) in the column:  ['Public on Mar 30 2020']
 
Column index:  3 ; Column name:  !Sample_submission_date
Count of unique values in the column:  1
The unique value(s) in the column:  ['Sep 09 2019']
 
Column index:  4 ; Column name:  !Sample_last_update_date
Count of unique values in the column:  1
The unique value(s) in the column:  ['Mar 30 2020']
 
Column index:  5 ; Column name:  !Sample_type
Count of unique values in the column:  1
The unique value(s) in the column:  ['RNA']
 
Column index:  6 ; Column name:  !Sample_channel_count
Count of unique values in the column:  1
The unique value(s) in the column: 

In the above, we would like to note the following statement (column # 22, !Sample_data_processing)in particular:

'The presence of miRNA was determined based on a corresponding microarray signal of greater than [the mean + 2 × standard deviation] of the negative controls signal, of which the top and bottom ranked ones by signal intensity were removed. Once a miRNA was considered present, the mean signal of the negative controls of which the top and bottom 5% ranked by signal intensity were removed was subtracted from the miRNA signal. When the signal value was negative (or undetEsophageal Cancerted) after background subtraction, the value was replaced by 0.1 on a base 2 logarithm scale.'


In [ ]:
# Here we remove the columns [0:8, 10:33] identified earlier.
# We do this in reverse order so that the identified column indices as deletion
#    targets, do not change as deletions are performed one at a time
for i in range(33, 9, -1):
  df1.drop(df1.columns[i], axis=1, inplace=True)
for i in range (8, -1, -1):
  df1.drop(df1.columns[i], axis=1, inplace=True)
#

In [ ]:
df1.head()
# Inspect to confirm correct deletion of columns

,!Sample_characteristics_1_ch1,MIMAT0000062,MIMAT0000063,MIMAT0000064,MIMAT0000065,MIMAT0000066,MIMAT0000067,MIMAT0000068,MIMAT0000069,MIMAT0000070,...,MIMAT0031893,MIMAT0032026,MIMAT0032029,MIMAT0032110,"MIMAT0032114, MIMAT0032115",MIMAT0032116,MIMAT0033692,MIMAT0035542,MIMAT0035703,MIMAT0035704
0,disease state: Non-cancer control,4.393092,6.306405,5.736186,3.923085,4.903123,-0.152745,-0.152745,-0.152745,3.958537,...,-0.152745,-0.152745,5.312396,-0.152745,-0.152745,7.239878,3.096798,-0.152745,-0.152745,0.252979
1,disease state: Non-cancer control,3.817744,-1.240936,3.740098,2.193073,2.147514,3.169063,3.485990,3.674215,5.033756,...,-1.240936,1.314120,6.919166,-1.240936,-1.240936,6.827426,4.358254,2.766291,-1.240936,-1.240936
2,disease state: Non-cancer control,3.779668,3.997013,0.503678,3.724792,1.606391,3.221160,2.860850,4.332943,4.093647,...,1.530600,-1.353647,6.632532,-1.531527,-1.531527,5.173705,4.844177,1.146725,-1.531527,-1.531527
3,disease state: Non-cancer control,5.537473,1.593298,3.843516,5.464836,0.530399,2.489050,3.275917,0.530399,5.951552,...,5.404108,0.530399,7.903956,4.832157,0.530399,6.444462,4.290578,0.530399,4.570456,4.113035
4,disease state: Non-cancer control,5.953741,3.073890,0.853141,4.977867,0.853141,6.203541,0.853141,0.853141,0.853141,...,0.853141,3.142655,8.361825,0.853141,0.853141,3.301567,0.853141,0.853141,0.853141,0.853141


In [ ]:
df1.info()
# Output:
#<class 'pandas.core.frame.DataFrame'>
#RangeIndex: 3924 entries, 0 to 3923
#Columns: 2566 entries, !Sample_characteristics_1_ch1 to MIMAT0035704
#dtypes: float64(2565), object(1)
#memory usage: 76.8+ MB
#
# Columns: 1 (for sample class label) + 2565 (for sample miRNA attributes/features)
# Rows: 3924 (for samples i.e., patients)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3924 entries, 0 to 3923
Columns: 2566 entries, !Sample_characteristics_1_ch1 to MIMAT0035704
dtypes: float64(2565), object(1)
memory usage: 76.8+ MB


In [ ]:
df1.shape
# (3924, 2566) : (row count, column count)

(3924, 2566)

In [ ]:
df1.columns
# Output:
#Index(['!Sample_characteristics_1_ch1', 'MIMAT0000062', 'MIMAT0000063',
#       'MIMAT0000064', 'MIMAT0000065', 'MIMAT0000066', 'MIMAT0000067',
#       'MIMAT0000068', 'MIMAT0000069', 'MIMAT0000070',
#       ...
#       'MIMAT0031893', 'MIMAT0032026', 'MIMAT0032029', 'MIMAT0032110',
#       'MIMAT0032114, MIMAT0032115', 'MIMAT0032116', 'MIMAT0033692',
#       'MIMAT0035542', 'MIMAT0035703', 'MIMAT0035704'],
#      dtype='object', length=2566)

Index(['!Sample_characteristics_1_ch1', 'MIMAT0000062', 'MIMAT0000063',
       'MIMAT0000064', 'MIMAT0000065', 'MIMAT0000066', 'MIMAT0000067',
       'MIMAT0000068', 'MIMAT0000069', 'MIMAT0000070',
       ...
       'MIMAT0031893', 'MIMAT0032026', 'MIMAT0032029', 'MIMAT0032110',
       'MIMAT0032114, MIMAT0032115', 'MIMAT0032116', 'MIMAT0033692',
       'MIMAT0035542', 'MIMAT0035703', 'MIMAT0035704'],
      dtype='object', length=2566)

In [ ]:
df1.index
# Output:
# RangeIndex(start=0, stop=3924, step=1)

RangeIndex(start=0, stop=3924, step=1)

In [ ]:
# Here we remove rows corresponding to samples i.e., patients, in the category
#    'disease state: Lung cancer, post-operation'
#    We do this again in reverse order, although strictly speaking this is not
#    necessary as we do not operate on predetermined row indices
#    We should be left with 3744 samples (based on Tarik Oguz's former work,
#    specifically, the Excel file we loaded above)
for i in range(3923, -1, -1):
  if df1.iloc[i,0] == 'disease state: Lung cancer, post-operation':
    df1.drop(df1.index[i], axis=0, inplace=True)
#

In [ ]:
df1.info()
# Output:
#<class 'pandas.core.frame.DataFrame'>
#RangeIndex: 3744 entries, 0 to 3743
#Columns: 2566 entries, !Sample_characteristics_1_ch1 to MIMAT0035704
#dtypes: float64(2565), object(1)
#memory usage: 73.3+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3744 entries, 0 to 3743
Columns: 2566 entries, !Sample_characteristics_1_ch1 to MIMAT0035704
dtypes: float64(2565), object(1)
memory usage: 73.3+ MB


In [ ]:
df1.shape
# (3744, 2566) : (# of rows, # of columns)

(3744, 2566)

In [ ]:
df1.columns
# Output:
#Index(['!Sample_characteristics_1_ch1', 'MIMAT0000062', 'MIMAT0000063',
#       'MIMAT0000064', 'MIMAT0000065', 'MIMAT0000066', 'MIMAT0000067',
#       'MIMAT0000068', 'MIMAT0000069', 'MIMAT0000070',
#       ...
#       'MIMAT0031893', 'MIMAT0032026', 'MIMAT0032029', 'MIMAT0032110',
#       'MIMAT0032114, MIMAT0032115', 'MIMAT0032116', 'MIMAT0033692',
#       'MIMAT0035542', 'MIMAT0035703', 'MIMAT0035704'],
#      dtype='object', length=2566)

Index(['!Sample_characteristics_1_ch1', 'MIMAT0000062', 'MIMAT0000063',
       'MIMAT0000064', 'MIMAT0000065', 'MIMAT0000066', 'MIMAT0000067',
       'MIMAT0000068', 'MIMAT0000069', 'MIMAT0000070',
       ...
       'MIMAT0031893', 'MIMAT0032026', 'MIMAT0032029', 'MIMAT0032110',
       'MIMAT0032114, MIMAT0032115', 'MIMAT0032116', 'MIMAT0033692',
       'MIMAT0035542', 'MIMAT0035703', 'MIMAT0035704'],
      dtype='object', length=2566)

In [ ]:
df1.index
# Output:
# RangeIndex(start=0, stop=3744, step=1)

RangeIndex(start=0, stop=3744, step=1)

In [ ]:
# This code cell is used to obtain the first six columns' names (labels) in df
#    to develop the code cell after the next for df1 to df comparison
df.head()

,!Sample_characteristics_ch1,MIMAT0000062,MIMAT0000063,MIMAT0000064,MIMAT0000065,MIMAT0000066,MIMAT0000067,MIMAT0000068,MIMAT0000069,MIMAT0000070,...,MIMAT0031893,MIMAT0032026,MIMAT0032029,MIMAT0032110,"MIMAT0032114, MIMAT0032115",MIMAT0032116,MIMAT0033692,MIMAT0035542,MIMAT0035703,MIMAT0035704
0,"disease state: Lung cancer, pre-operation",3.348626,5.804969,0.593266,4.590442,0.593266,4.310643,0.593266,4.277008,2.534327,...,0.593266,0.593266,7.236875,0.593266,0.593266,6.769631,0.593266,0.593266,0.593266,0.593266
1,"disease state: Lung cancer, pre-operation",5.857715,5.735498,4.275580,5.630426,4.770349,5.821116,2.598021,5.651709,3.469541,...,6.195828,0.250025,6.929795,0.250025,0.250025,6.708787,5.673383,5.347314,0.250025,0.250025
2,"disease state: Lung cancer, pre-operation",6.368630,4.831915,5.763215,4.885079,2.452500,5.763215,0.385688,6.174210,5.455496,...,0.385688,0.385688,8.090484,0.385688,0.385688,7.267933,3.238758,2.107493,0.385688,3.901948
3,"disease state: Lung cancer, pre-operation",6.653868,7.545660,6.231679,6.782832,1.328411,5.948545,3.979502,6.657316,5.438513,...,1.049994,1.049994,8.465234,1.049994,1.049994,5.787249,4.767387,5.079382,1.049994,5.056768
4,"disease state: Lung cancer, pre-operation",4.850008,5.095308,3.874733,6.019523,0.364696,4.143121,0.364696,5.260521,0.364696,...,0.364696,0.364696,8.337157,0.364696,2.667634,7.981677,5.024807,4.695774,0.364696,0.364696


In [ ]:
df.columns

Index(['!Sample_characteristics_ch1', 'MIMAT0000062', 'MIMAT0000063',
       'MIMAT0000064', 'MIMAT0000065', 'MIMAT0000066', 'MIMAT0000067',
       'MIMAT0000068', 'MIMAT0000069', 'MIMAT0000070',
       ...
       'MIMAT0031893', 'MIMAT0032026', 'MIMAT0032029', 'MIMAT0032110',
       'MIMAT0032114, MIMAT0032115', 'MIMAT0032116', 'MIMAT0033692',
       'MIMAT0035542', 'MIMAT0035703', 'MIMAT0035704'],
      dtype='object', length=2566)

In [ ]:
# Here, we will compare what we have prepared in df1, to what was prepared and
#    used by Tarik Oguz through the original Excel file loaded into df
# There is possibly the need for one more step of preprocessing on df1 as the
#    data in df is sorted with respect to class labels with disease state: Lung cancer, pre-operation
#    samples first, and then the disease state: Non-cancer control samples, whereas
#    the data in df1 is somewhat mixed in this sense, hence a direct, ordered
#    row-by-row comparison is not feasible
# Since, exactly through what kind of Excel implemented permutations df has been
#    sorted, is not known, for an in-place comparison of the data in df and df1,
#    we will take the approach of first (re)sorting both df and df1 in exactly the
#    same manner using their first text and the following five numeric formatted
#    columns, which is deemed to be sufficient to achieve identical sorting
#    results for both dataframes. We will consequently perform an in-place comparison
#    of the numeric fields to confirm identicallity.
#
# First, multisort df using its first 6 columns
df.sort_values(by=['!Sample_characteristics_ch1', 'MIMAT0000062', 'MIMAT0000063', 'MIMAT0000064',  'MIMAT0000065',  'MIMAT0000066'], inplace=True)
#
# Now, multisort df1 in exacly the same manner
df1.sort_values(by=['!Sample_characteristics_1_ch1', 'MIMAT0000062', 'MIMAT0000063', 'MIMAT0000064',  'MIMAT0000065',  'MIMAT0000066'], inplace=True)
#
# Rename the column in df1 to match df for comparison, this is a pandas dataframe .compare() utility requirement
df1.rename(columns={'!Sample_characteristics_1_ch1': '!Sample_characteristics_ch1'}, inplace=True)
#
# Reset indeces for both dataframes df and df1 to ensure identical indices for comparison,
#    most likely multisorting also leads to a permutation of row indices and this breaks .compare() utility
df.reset_index(drop=True, inplace=True)
df1.reset_index(drop=True, inplace=True)

print(df1.equals(df))
# Output: False
# In the following, in spite of the 'False' .equals() utility output, we will
#    establish that the two dataframes df and df1 are actually identical within
#    a very small (1e-20) numerical tolerance

False


In [ ]:
print(df1.compare(df))
#
# First, let's establish that there are a very substantial number of cells (728x1458) in
#    df and df1 which are identical, as identified by a NaN output from the
#    .compare() utility
#
# When comparing two pandas DataFrames and receiving NaN as output, ....
#    Specifically, NaN (Not a Number) in this context indicates one of the following:
# Matched Values in df.compare(): If you are using the df.compare() method, NaN
#    in the output DataFrame signifies that the values in the corresponding
#    positions of the two input DataFrames are identical. The compare() method
#    highlights differences, so if there's no difference (i.e., the values are
#    the same), it often represents this with NaN in the output, especially if
#    keep_shape=True and keep_equal=False (which are the defaults for compare()
#    in many scenarios).
#
#    import pandas as pd
#    import numpy as np
#
#    df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
#    df2 = pd.DataFrame({'A': [1, 2, 99], 'B': [4, 5, 6]})
#
#    # Comparing df1 and df2
#    comparison = df1.compare(df2)
#    print(comparison)
#
# In this output, NaN would appear where the values in 'A' and 'B' are the same
#    between df1 and df2. The only non-NaN value would be in 'A' where 3 and 99 differ.
# ...
# In summary, when NaN appears during DataFrame comparison, it primarily indicates
#    either that the values being compared are identical when using df.compare(),
#    or that a value was missing during a merge/join operation.
#

     MIMAT0000062       MIMAT0000063       MIMAT0000067       MIMAT0000068  \
             self other         self other         self other         self   
3             NaN   NaN          NaN   NaN          NaN   NaN          NaN   
5             NaN   NaN          NaN   NaN          NaN   NaN          NaN   
16            NaN   NaN          NaN   NaN          NaN   NaN          NaN   
20            NaN   NaN          NaN   NaN          NaN   NaN          NaN   
23            NaN   NaN          NaN   NaN          NaN   NaN          NaN   
...           ...   ...          ...   ...          ...   ...          ...   
3685          NaN   NaN          NaN   NaN          NaN   NaN          NaN   
3690          NaN   NaN          NaN   NaN          NaN   NaN          NaN   
3701          NaN   NaN          NaN   NaN          NaN   NaN          NaN   
3702          NaN   NaN          NaN   NaN          NaN   NaN          NaN   
3712          NaN   NaN          NaN   NaN          NaN   NaN   

In [ ]:
# Now we establish that actually ALL numeric cells of df and df1 are equal within
#    a very small absolute tolerance given by 1e-20
#
# Using pandas.testing.assert_frame_equal:
# This function is designed for testing and offers built-in tolerance options.
# Key parameters for tolerance:
# atol (absolute tolerance): The maximum absolute difference between two values for them to be considered equal.
# rtol (relative tolerance): The maximum allowed difference relative to the magnitude of the second value.
# When using assert_frame_equal, set check_exact=False to enable tolerance-based comparison and specify rtol or atol.
#
#import pandas as pd
from pandas.testing import assert_frame_equal

try:
    assert_frame_equal(df1, df, check_exact=False, atol=1e-20) # or rtol
    print("DataFrames are equal within tolerance.")
except AssertionError as e:
    print(f"DataFrames are not equal within tolerance: {e}")

DataFrames are equal within tolerance.


In [ ]:
# [1] The microarray used for the original study represented xxxx miRNAs.
print('The microarray used for the original study represented', df1.shape[1]-1,'miRNAs.')
# Save the complete list of identifiers (labels) for the miRNAs used in the
#    original study to a csv file.
#df1.columns[1:].to_csv('/content/miRNA_identifiers.csv', index=False) # incorrect, index object .columns[1:] does not have a .to_csv() method
#print(df1.columns[1:]) # works as expected
# 1. Get the column labels
column_labels = df1.columns[1:]
# 2. Convert to a list (optional)
column_labels_list = column_labels.tolist()
# 3. Create a new DataFrame from the column labels
## Option 1: Create a DataFrame with a default column name (0)
#df_column_labels_default = pd.DataFrame(column_labels)
#print("\nDataFrame of column labels (default column name):")
#print(df_column_labels_default)
# Option 2: Create a DataFrame with a specified column name
df_column_labels_named = pd.DataFrame(column_labels, columns=['Identifiers_of_all_miRNAs_used_in_the_original_study'])
#print("\nDataFrame of column labels (named column):")
#print(df_column_labels_named)
#
## Write the DataFrame to a CSV file
#df.to_csv('output.csv', index=False)
## 'output.csv' is the desired filename.
## index=False prevents writing the DataFrame index as a column in the CSV.
df_column_labels_named.to_csv('/content/miRNA_identifiers.csv', index=False)

The microarray used for the original study represented 2565 miRNAs.


Beyond this point is the original project data analysis implementation of Tarik Oguz used for ISEF.